<a href="https://colab.research.google.com/github/Nemoy2897/Chatbot286/blob/master/Training_a_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import time
import itertools

import tensorflow as tf

In [2]:
ls

sample_data/


In [7]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving movie_conversations.txt to movie_conversations.txt
Saving movie_lines.txt to movie_lines.txt
User uploaded file "movie_conversations.txt" with length 6760930 bytes
User uploaded file "movie_lines.txt" with length 34641919 bytes


In [0]:
# !git clone https://github.com/suriyadeepan/datasets/blob/master/seq2seq/cornell_movie_corpus/scripts/prepare_data.py

In [5]:
import sys
import os
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('punkt')
from keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense, Bidirectional
from keras.models import Model, load_model

INPUT_LENGTH = 20
OUTPUT_LENGTH = 20

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using TensorFlow backend.


#1.Opening Files with utf encoding 

In [0]:
lines = open ('movie_lines.txt',encoding = 'utf-8', errors = 'ignore').read().split('\n')
conv_lines = open('movie_conversations.txt', encoding = 'utf-8',errors = 'ignore').read().split('\n')

# 2. Mapping Line id with dialogues

In [0]:
id2line = {}
for line in lines:
  _line = line.split(' +++$+++ ')
  if len(_line) == 5:
    id2line[_line[0]]=_line[4]

In [0]:
convs = []
for line in conv_lines[:-1]:
    _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    convs.append(_line.split(','))

In [11]:
for k in convs[298]:
    print (k, id2line[k])

L3464 ...delusions and paranoia.
L3465 I was all of these.
L3466 Well, you didn't appreciate the severity of it until recently.  No question about that.
L3467 What about Oleg?
L3468 Disappeared.  They're looking everywhere.  Maybe he went back to Czechoslovakia.
L3469 No, he is here.  Shit...
L3470 Don't worry about him.  Think about yourself.
L3471 What about my movie rights?  Book rights?
L3472 Look, I haven't really focused on that kind of thing.
L3473 What's your cut?  How much?
L3474 I would say...half.  Half is fair.
L3475 No.  No way.
L3476 But it's...
L3477 Thirty-percent.  No more.  Or I call another lawyer.  This is the biggest case of your life.  Don't try to negotiate.  Thirty percent.  Say yes or no.
L3478 This is not about money, Emil.  I need your trust in me.
L3479 What else do you need?
L3480 I need to know about your background.  I need to know about your upbringing.  Why you're here.
L3481 Give me another one, please.


# 3. Splitting Dialogues to Q & A

In [12]:
questions = []
answers = []
for conv in convs:
  for i in range(len(conv)-1):
    questions.append(id2line[conv[i]])
    answers.append(id2line[conv[i+1]])
    
print(len(questions))
print(len(answers))

221616
221616


# 4. Clean the text and remove the shorthands and Symbols

In [0]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|]", "", text)
    text = " ".join(text.split())
    return text

In [0]:
# Clean the data
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))
clean_answers = []    
for answer in answers:
    clean_answers.append(clean_text(answer))

In [15]:
lengths = []

for question in clean_questions:
  lengths.append(len(question.split()))
for answer in clean_answers:
  lengths.append(len(answer.split()))
  
lengths= pd.DataFrame(lengths, columns = ['counts'])
print(np.percentile(lengths,80))
print(np.percentile(lengths,75))
print(np.percentile(lengths,90))
print(np.percentile(lengths,95))

16.0
14.0
24.0
32.0


# 5. Removing too long and too short Q & A

In [16]:
# Remove questions and answers that are shorter than 1 word and longer than 20 words.
min_line_length = 2
max_line_length = 20

# Filter out the questions that are too short/long
short_questions_temp = []
short_answers_temp = []

for i,question in enumerate(clean_questions):
  if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
    short_questions_temp.append(question)
    short_answers_temp.append(clean_answers[i])

# Filter out the answers that are too short/long
short_questions = []
short_answers = []

for i, answer in enumerate(short_answers_temp):
    if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
        short_answers.append(answer)
        short_questions.append(short_questions_temp[i])
        
print(len(short_questions))
print(len(short_answers))


138528
138528


#6. Pre processing data for word model


In [17]:
num_samples  = 30000 #no. of samples to train on
short_questions = short_questions[:num_samples]
short_answers = short_answers[:num_samples]

#tokenizing the ques and ans
short_ques_tok = [nltk.word_tokenize(sent) for sent in short_questions] 
short_ans_tok = [nltk.word_tokenize(sent) for sent in short_answers]

data_size = len(short_ques_tok)

training_input = short_ques_tok[:round(data_size*(0.8))]
training_input = [tr_input[::-1] for tr_input in training_input]

training_output = short_ans_tok[:round(data_size*0.8)]

validation_input = short_ques_tok[round(data_size*(80/100)):]
validation_input  = [val_input[::-1] for val_input in validation_input] #reverseing input seq for better performance
validation_output = short_ans_tok[round(data_size*(80/100)):]

print('training size', len(training_input))
print('validation size', len(validation_input))

training size 24000
validation size 6000


In [0]:
from nltk.probability import FreqDist
import itertools
freqdist = FreqDist(itertools.chain(*(short_ques_tok+short_ans_tok)))
 

  

In [0]:
vocab =dict( freqdist.most_common(1938))

In [20]:
print(vocab)

{'.': 55097, 'you': 24017, ',': 22569, 'i': 21936, '?': 21324, 'the': 12485, 'is': 12190, 'not': 12160, 'to': 10352, 'it': 9977, 'a': 9334, 'do': 8046, 'that': 7769, '...': 6802, '!': 6299, 'are': 6282, 'what': 5770, 'me': 4984, 'have': 4899, 'of': 4662, 'and': 4271, 'in': 4263, 'am': 4187, 'we': 4170, 'he': 3781, 'will': 3451, 'know': 3336, 'this': 3335, 'for': 3295, 'your': 3211, 'no': 3137, 'my': 3125, 'was': 2917, "'s": 2879, 'can': 2792, 'be': 2706, 'on': 2587, 'did': 2520, 'just': 2474, 'would': 2281, 'like': 2202, 'with': 2192, 'but': 2120, 'here': 2076, 'about': 2068, 'they': 2038, 'how': 2028, 'get': 1970, 'there': 1937, 'all': 1924, 'want': 1854, 'so': 1848, 'got': 1782, 'think': 1751, 'if': 1670, 'out': 1668, 'well': 1657, 'go': 1621, 'she': 1620, 'oh': 1590, 'him': 1586, 'why': 1546, 'right': 1518, 'up': 1480, 'now': 1438, 'going': 1368, 'where': 1329, 'one': 1312, 'at': 1311, 'yes': 1245, 'who': 1221, 'see': 1209, 'yeah': 1203, 'good': 1113, 'her': 1111, 'come': 1018, 'say

In [21]:
#we will create dictionaries to provide a unique integer for each word.
WORD_CODE_START = 1
WORD_CODE_PADDING = 0

word_num =2
encoding  = {}
decoding = {1:'START'}
threshold =15
for word, count in vocab.items():
  if count >= threshold:
    encoding[word] = word_num
    decoding[word_num] =word
    word_num+=1

print("No. of vocab used:", word_num)

No. of vocab used: 1940


In [0]:
decoding[len(encoding)+2]='<UNK>'
encoding['<UNK>'] = len(encoding)+2

In [0]:
dict_size = word_num +1

# 7. vectoring dataset

In [0]:
def transform(encoding, data, vector_size =20):
  transformed_data = np.zeros(shape=(len(data), vector_size))
  for i in range(len(data)):
    for j in range(min(len(data[i]), vector_size)):
      try:
        transformed_data[i][j] = encoding[data[i][j]]
      except:
        transformed_data[i][j] = encoding['<UNK>']
  return transformed_data

In [25]:
enc_train_input = transform(encoding, training_input,vector_size= INPUT_LENGTH)

enc_train_output = transform(encoding, training_output,vector_size=OUTPUT_LENGTH)

print('encoded training input', enc_train_input.shape)
print('encoded training output', enc_train_output.shape)

encoded training input (24000, 20)
encoded training output (24000, 20)


In [26]:
enc_valid_input =  transform(encoding, validation_input, vector_size=INPUT_LENGTH)

enc_valid_output = transform(encoding,validation_output,vector_size = OUTPUT_LENGTH)

print('encoded validation input', enc_valid_input.shape)
print('encoded validation output', enc_valid_output.shape)

encoded validation input (6000, 20)
encoded validation output (6000, 20)


#Model Building
##Sequence to Sequence Models

In [0]:
tf.keras.backend.clear_session()

In [0]:
INPUT_LENGTH =20
OUTPUT_LENGTH =20

enc_input = Input(shape=(INPUT_LENGTH,))
dec_input = Input(shape=(OUTPUT_LENGTH,))
  

In [29]:
from keras.layers import SimpleRNN

encoder = Embedding(dict_size, 128, input_length=INPUT_LENGTH, mask_zero=True)(enc_input)

encoder = LSTM(512, return_sequences = True, unroll= True)(encoder)

encoder_last = encoder[:,-1,:]

print('encoder', encoder)
print('encoder_last', encoder_last)

decoder = Embedding(dict_size, 128, input_length=OUTPUT_LENGTH, mask_zero=True)(dec_input)

decoder = LSTM(512, return_sequences = True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])

print('decoder', decoder)

# For the plain Sequence-to-Sequence, we produced the output from directly from decoder
# output = TimeDistributed(Dense(output_dict_size, activation="softmax"))(decoder)

encoder Tensor("lstm_1/transpose_2:0", shape=(?, 20, 512), dtype=float32)
encoder_last Tensor("strided_slice:0", shape=(?, 512), dtype=float32)
decoder Tensor("lstm_2/transpose_2:0", shape=(?, 20, 512), dtype=float32)


#Attention Mechanism

In [30]:
from keras.layers import Activation, dot, concatenate

attention = dot([decoder,encoder], axes = [2,2])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

context = dot([attention, encoder], axes =[2,1])
print('context', context)

decoder_combined_context= concatenate([context,decoder])
print('decoder_combined_context', decoder_combined_context)

output = TimeDistributed(Dense(512, activation='tanh'))(decoder_combined_context)
output = TimeDistributed(Dense(dict_size, activation='softmax'))(output)
print('output', output)


attention Tensor("attention/truediv:0", shape=(?, 20, 20), dtype=float32)
context Tensor("dot_2/MatMul:0", shape=(?, 20, 512), dtype=float32)
decoder_combined_context Tensor("concatenate_1/concat:0", shape=(?, 20, 1024), dtype=float32)
output Tensor("time_distributed_2/Reshape_1:0", shape=(?, 20, 1941), dtype=float32)


In [31]:
model= Model(inputs=[enc_input, dec_input], outputs=[output])
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 20, 128)      248448      input_2[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 128)      248448      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LS

In [0]:
train_enc_input = enc_train_input 
train_dec_input = np.zeros_like(enc_train_input)
train_dec_input[:,1:] = enc_train_output[:,:-1]
train_dec_input[:,0] = WORD_CODE_START

train_dec_output=np.eye(dict_size)[enc_train_output.astype('int')]

valid_enc_input = enc_valid_input
valid_dec_input = np.zeros_like(enc_valid_output)
valid_dec_input[:,1:] = enc_valid_output[:,:-1]
valid_dec_input[:,0] = WORD_CODE_START
valid_dec_output = np.eye(dict_size)[enc_valid_output.astype('int')]



In [33]:
model.fit(x=[train_enc_input, train_dec_input], y=[train_dec_output],
         validation_data =([valid_enc_input, valid_dec_input], [valid_dec_output]),
         batch_size=64, epochs=100)



Train on 24000 samples, validate on 6000 samples
Epoch 1/100
24000/24000 [==============================] - 64s 3ms/step - loss: 0.0031 - val_loss: 0.0028
Epoch 2/100
24000/24000 [==============================] - 54s 2ms/step - loss: 0.0027 - val_loss: 0.0026
Epoch 3/100
24000/24000 [==============================] - 54s 2ms/step - loss: 0.0025 - val_loss: 0.0025
Epoch 4/100
24000/24000 [==============================] - 54s 2ms/step - loss: 0.0024 - val_loss: 0.0024
Epoch 5/100
24000/24000 [==============================] - 54s 2ms/step - loss: 0.0024 - val_loss: 0.0024
Epoch 6/100
24000/24000 [==============================] - 54s 2ms/step - loss: 0.0023 - val_loss: 0.0024
Epoch 7/100
24000/24000 [==============================] - 54s 2ms/step - loss: 0.0023 - val_loss: 0.0023
Epoch 8/100
24000/24000 [==============================] - 54s 2ms/step - loss: 0.0023 - val_loss: 0.0023
Epoch 9/100
24000/24000 [==============================] - 54s 2ms/step - loss: 0.0023 - val_loss: 0.00

In [0]:
def prediction(raw_input):
  clean_input = clean_text(raw_input)
  input_tok = [nltk.word_tokenize(clean_input)]
  input_tok = [input_tok[0][::-1]] #reversing input sequence
  encoder_input = transform(encoding, input_tok,20)
  decoder_input = np.zeros(shape=(len(encoder_input), OUTPUT_LENGTH))
  decoder_input[:,0] = WORD_CODE_START
  for i in range(1, OUTPUT_LENGTH):
    output = model.predict([encoder_input,decoder_input]).argmax(axis=2)
    decoder_input[:,i] = output[:,i]
  return output

def decode(decoding, vector):
  text = ''
  for i in vector:
    if i==0:
      break
    text += ' '
    text+= decoding[i]
    
  return text

In [35]:
for i in range(20):
  seq_index = np.random.randint(1, len(short_questions))
  output = prediction(short_questions[seq_index])
  print('Q: ', short_questions[seq_index])
  print('A: ', decode(decoding, output[0]))

Q:  no one to back me up. now i have a good idea. so unless you come up with something better
A:   dignan . i can not do that . all right ? i just can not .
Q:  how? what do they do for you, kaminski and his friends? they are all anarchists! traitors!
A:   no , <UNK> . it is just <UNK> . some <UNK> <UNK> also . i am just
Q:  well, do not do that again!
A:   how are we going to live , <UNK> ? do you want me to go into me on ? ?
Q:  you are happy for me and victor.
A:   i can tell you guys are <UNK> what was the best part of our relationship
Q:  no there had to be someone else. who knew?!
A:   only me and dr. <UNK> .
Q:  and why was the alarm on?
A:   i could not just the fire <UNK> control at the other .
Q:  barak is still strong in the north, and tendo holds the high passes. but the great desert forts have fallen.
A:   oh . you seem about his <UNK> .
Q:  if you say you are.
A:   i most definitely say i am .
Q:  he is out.
A:   i know he is out , but how is he doing ?
Q:  it is just not

In [40]:
for i in range(5):
  seq=input()
  output = prediction(seq)
  print('A: ', decode(decoding, output[0]))


my birthday is 28 jan 1997, how long do  I live?
A:   four years .
Inconsistent results yeah?
A:   you are the <UNK> for east <UNK> , o'neil .
my name is not o'neil
A:   something new .
something new
A:   oh , thank you . have i <UNK> you ?
fuck you
A:   just a thought .
